The latest version of Gurobi (v10.0) introduced the matrix variable (Mvar) type. It is a very useful tool for modeling. However, it is not easy to access the Mvar like getVarByName() from a outside function. In this notebook, I will show you how to access the Mvar in the outside of the model.

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

This sample is to change the value np.ndarry parameter "b" before solving the problem.
To achieve this, we need to fix the value of a Mvar (b) by using the external function "_fix_mvar"

In [2]:
model = gp.Model("LP")

x = model.addMVar(2, lb=0, ub=gp.GRB.INFINITY)

c = np.array([[3, 5]])
A = np.array([[1, 0],
                [0, 2],
                [3, 2]])

# b_old = np.array([4, 12, 18]) # The optimal value should be 36
# b_new = np.array([0.5, 0.5, 0.5]) # The optimal value should be 1.25
b = model.addMVar(shape=3,name="b")

model.addConstr(A @ x <= b)
# m.addConstr(b == np.array([4, 12, 18]))

model.setObjective(c @ x, GRB.MAXIMIZE)

model.update() #NOTE !! You MUST call update() method before modifying the model


In [3]:
def _getMvarByName(model:gp.Model,mvar_name:str,shape:list) -> dict:
    """Mar version of getVarByName in gurobi 10.0

    Args:
        model (grb.Model): gurobi model
        mvar_name (str): mvar name defined in gurobi model
        dim (list): dimension of mvar. For 1D mvar, dim = [i]. For 2D mvar, dim = [i,j]

    Returns:
        dict: a dictionary of mvar, which links the original mvar name to a new name that can be used in external functions
    """
    mvars_ = {}
    if len(shape) == 1:
        for i in range(shape[0]):
            mvars_[i] = model.getVarByName(mvar_name + "[%d]" % (i))    
                 
    elif len(shape) == 2:
        for i in range(shape[0]):
            for j in range(shape[1]):
                mvars_[i,j] = model.getVarByName(mvar_name + "[%d,%d]" % (i,j))  
                      
    else:
        raise ValueError("Currently only 1D and 2D mvars are supported")
        
    return mvars_


def _fixMvar(model:gp.Model, mvar_name:str, shape:list, value:np.ndarray, cons_name:str) -> None:
    
    dict_mvar = _getMvarByName(model, mvar_name, shape)
    if value.ndim == 1:
        if model.getConstrByName(cons_name + "[0]") is not None:
            _removeMvarConstrs(model, cons_name, shape)
            
        model.addConstrs((dict_mvar[i] == value[i] for i in range(len(value))), name=cons_name)
        
    elif value.ndim == 2:
        if model.getConstrByName(cons_name + "[0,0]") is not None:
            _removeMvarConstrs(model, cons_name, shape)
            
        model.addConstrs((dict_mvar[i,j] == value[i,j] for i in range(value.shape[0]) for j in range(value.shape[1])), name=cons_name)
        
    else:
        raise ValueError("Currently only 1D and 2D mvars are supported")
    
    
def _removeMvarConstrs(model:gp.Model, cons_name:str, shape:list) -> None:
    
    cons = {}
    if len(shape) == 1:
        for i in range(shape[0]):
            cons[i] = model.getConstrByName(cons_name + "[%d]" %(i))
            
    elif len(shape) == 2:
        for i in range(shape[0]):
            for j in range(shape[1]):
                cons[i, j] = model.getConstrByName(cons_name + "[%d,%d]" %(i, j))
    else:
        raise ValueError("Currently only 1D and 2D mvars are supported")
            
    model.remove(cons)

In [4]:
b_old = np.array([4, 12, 18])    
_fixMvar(model=model,mvar_name="b",shape=[3],value=b_old,cons_name="fix_b")
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i5-1340P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x9ddcd0ca
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 5 rows and 3 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   1.500000e+00   0.000000e+00      0s
       1    3.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.600000000e+01


In [5]:
b_new = np.array([0.5, 0.5, 0.5])
_fixMvar(model=model,mvar_name="b",shape=[3],value=b_new, cons_name="fix_b")
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i5-1340P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x78093ce5
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 5e-01]
Presolve removed 6 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
       0    1.2500000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.250000000e+00
